##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Neural XOR

## Install and import dependencies

We'll need libraries.

In [0]:
from __future__ import absolute_import, division, print_function


import torch

import torch.nn as nn

import torch.nn.functional as F

import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt



In [0]:
def create_example(N, batch_size):
  A = np.random.binomial(n=1, p=0.5, size=(batch_size, N))
  B = np.random.binomial(n=1, p=0.5, size=(batch_size, N,))

  X = np.zeros((batch_size, 2 *N,), dtype=np.float32)
  X[:,:N], X[:,N:] = A, B

  Y = (A ^ B).astype(np.float32)
  return X,Y

In [0]:
X, Y = create_example(3, 2)
print (X[0,:6] , "xor =", Y[0])


# Hyperparameters

In [0]:
N=5

batch_size = 32

batches_no = 500

hidden_units = N*20

lr = 0.001

epochs_no = 5000

### Test


In [0]:
class XORDataset(Dataset):

  def __init__(self, N, batches_no):

    self.N = N

    self.batches_no = batches_no

      

  def __len__(self):

    return self.batches_no
  
  def __getitem__(self, item):

    A=np.random.binomial( n= 1, p=0.5, size=N )

    B=np.random.binomial( n= 1, p=0.5, size=N )

    X = np.zeros(2*N, dtype=np.float32)

    X[:N], X[N:] = A , B

    Y = (A ^ B).astype(np.float32)

    return X, Y

In [0]:
class XOR_NN(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):

      super(XOR_NN, self).__init__()

      self.fc1 = nn.Linear(input_dim, hidden_dim)

      self.fc2 = nn.Linear(hidden_dim, output_dim)



    def forward(self, x_in):

      a_1 = F.relu(self.fc1(x_in))

      y_pred = torch.sigmoid(self.fc2(a_1))

      return y_pred



# Accuracy

def get_accuracy(y_pred, y_target):

  n_correct = torch.eq(y_pred.round(), y_target).sum().item()

  accuracy = n_correct / len(y_pred) * 100 / N

  return accuracy



# data

dataset = XORDataset(N, batches_no)

dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=6)



# Initialize model

model = XOR_NN(input_dim=2*N, hidden_dim=hidden_units, output_dim=N)

loss_fn = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=lr) 
# Adam optimizer (usually better than SGD)



# Training

for t in range(epochs_no):

  for X_train, y_train in dataloader:

    # Forward pass

    y_pred = model(X_train)

    

    # Accuracy

    accuracy = get_accuracy(y_pred=y_pred, y_target=y_train)



    # Loss

    loss = loss_fn(y_pred, y_train)

    

    # Verbose

    if t%100==0:

        print ("epoch: {0:02d} | loss: {1:.4f} | acc:{2:.1f}%".format(t, loss, accuracy))

        print("pred-target:", y_pred[0], y_train[0])



    # Zero all gradients

    optimizer.zero_grad()



    # Backward pass

    loss.backward()



    # Update weights

    optimizer.step()
